In [ ]:
import pandas as pd
chunk_iter = pd.read_csv('../data/raw/complaints.csv', chunksize=10000)
first_chunk = next(chunk_iter)

In [5]:
print(first_chunk.head())  
print(first_chunk.shape)   
print(first_chunk.columns)

  Date received                                            Product  \
0    2025-06-20  Credit reporting or other personal consumer re...   
1    2025-06-20                                    Debt collection   
2    2025-06-20  Credit reporting or other personal consumer re...   
3    2025-06-20  Credit reporting or other personal consumer re...   
4    2025-06-20  Credit reporting or other personal consumer re...   

               Sub-product                                 Issue  \
0         Credit reporting  Incorrect information on your report   
1  Telecommunications debt     Attempts to collect debt not owed   
2         Credit reporting           Improper use of your report   
3         Credit reporting           Improper use of your report   
4         Credit reporting  Incorrect information on your report   

                                       Sub-issue Consumer complaint narrative  \
0            Information belongs to someone else                          NaN   
1       

In [7]:
product_counts = first_chunk['Product'].value_counts()
print(product_counts)


Product
Credit reporting or other personal consumer reports        9608
Debt collection                                             290
Credit card                                                  42
Checking or savings account                                  20
Money transfer, virtual currency, or money service           20
Payday loan, title loan, personal loan, or advance loan       7
Vehicle loan or lease                                         6
Mortgage                                                      3
Debt or credit management                                     2
Prepaid card                                                  1
Student loan                                                  1
Name: count, dtype: int64


In [ ]:
first_chunk['word_count'] = first_chunk['Consumer complaint narrative'].str.split().str.len()
print(first_chunk['word_count'].describe())

count      2.000000
mean     382.000000
std       91.923882
min      317.000000
25%      349.500000
50%      382.000000
75%      414.500000
max      447.000000
Name: word_count, dtype: float64


In [9]:
print(first_chunk['Consumer complaint narrative'].isna().sum()) 

9998


In [57]:
import pandas as pd

# Load raw data (if not already loaded)
full_data = pd.read_csv('../data/raw/complaints.csv')
usable_data = full_data[['Product', 'Issue']].dropna()

# Print initial stats
print(f"Total usable rows: {len(usable_data)}")
print("\nTop Products:")
print(usable_data['Product'].value_counts().head(10))

print("="*50)  # Visual separator

# Filter for key products
key_products = [
    "Credit card", 
    "Checking or savings account",
    "Money transfer, virtual currency, or money service"
]
filtered_data = usable_data[usable_data['Product'].isin(key_products)].copy()
print(f"Filtered rows for key products: {len(filtered_data)}")

# Add loan-related complaints using keyword search
loan_products = ["Payday loan", "Personal loan", "Student loan"]
filtered_data = pd.concat([
    usable_data[usable_data['Product'].isin(key_products)],
    usable_data[usable_data['Product'].str.contains('|'.join(loan_products), case=False)]
])
print(f"Total rows after adding loan-related complaints: {len(filtered_data)}")

print("="*50)
print("Final Product Distribution:")
print(filtered_data['Product'].value_counts())

C:\Users\Simbo\AppData\Local\Temp\ipykernel_72120\145183388.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  full_data = pd.read_csv('../data/raw/complaints.csv')


Total usable rows: 9609791

Top Products:
Product
Credit reporting or other personal consumer reports                             4834854
Credit reporting, credit repair services, or other personal consumer reports    2163857
Debt collection                                                                  799197
Mortgage                                                                         422254
Checking or savings account                                                      291173
Credit card                                                                      226686
Credit card or prepaid card                                                      206369
Money transfer, virtual currency, or money service                               145066
Credit reporting                                                                 140429
Student loan                                                                     109717
Name: count, dtype: int64
Filtered rows for key products: 662925
Total

In [60]:
filtered_data['clean_text'] = filtered_data['Issue'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
filtered_data.to_csv('../data/filtered_complaints.csv', index=False)